##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/validate_correctness"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

# Validar la corrección y la equivalencia numérica

Cuando migre su código TensorFlow de TF1.x a TF2, es recomendable asegurarse de que el código migrado se comporta de la misma manera en TF2 que en TF1.x.

Esta guía cubre ejemplos de código de migración con el shim de modelado `tf.compat.v1.keras.utils.track_tf1_style_variables` aplicado a los métodos `tf.keras.layers.Layer`. Lea la [guía de mapeo de modelos](./model_mapping.ipynb) para saber más sobre los shim de modelado de TF2.

Esta guía detalla los enfoques que puede usar para:

- Validar la corrección de los resultados obtenidos de los modelos de entrenamiento usando el código migrado.
- Valide la equivalencia numérica de su código en todas las versiones de TensorFlow

## Preparación

In [ ]:
!pip uninstall -y -q tensorflow

In [ ]:
# Install tf-nightly as the DeterministicRandomTestTool is available only in
# Tensorflow 2.8
!pip install -q tf-nightly

In [ ]:
!pip install -q tf_slim

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1

import numpy as np
import tf_slim as slim
import sys


from contextlib import contextmanager

In [ ]:
!git clone --depth=1 https://github.com/tensorflow/models.git
import models.research.slim.nets.inception_resnet_v2 as inception

Si va a introducir en el shim un trozo no trivial de código de pasada hacia delante, querrá saber que se comporta de la misma manera que en TF1.x. Por ejemplo, considere la posibilidad de intentar introducir en el shim un modelo completo de TF-Slim Inception-Resnet-v2 como tal:

In [ ]:
# TF1 Inception resnet v2 forward pass based on slim layers
def inception_resnet_v2(inputs, num_classes, is_training):
  with slim.arg_scope(
    inception.inception_resnet_v2_arg_scope(batch_norm_scale=True)):
    return inception.inception_resnet_v2(inputs, num_classes, is_training=is_training)

In [ ]:
class InceptionResnetV2(tf.keras.layers.Layer):
  """Slim InceptionResnetV2 forward pass as a Keras layer"""

  def __init__(self, num_classes, **kwargs):
    super().__init__(**kwargs)
    self.num_classes = num_classes

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    is_training = training or False 
    
    # Slim does not accept `None` as a value for is_training,
    # Keras will still pass `None` to layers to construct functional models
    # without forcing the layer to always be in training or in inference.
    # However, `None` is generally considered to run layers in inference.
    
    with slim.arg_scope(
        inception.inception_resnet_v2_arg_scope(batch_norm_scale=True)):
      return inception.inception_resnet_v2(
          inputs, self.num_classes, is_training=is_training)


Resulta que, en realidad, esta capa funciona a la perfección recién incorporada (incluso tiene un seguimiento preciso de las pérdidas por regularización).

Sin embargo, no debe darlo por sentado. Siga los pasos que se indican a continuación para verificar que realmente se comporta como en TF1.x, hasta observar una equivalencia numérica perfecta. Estos pasos también pueden ayudarle a triangular qué parte de la pasada hacia delante está causando una divergencia con respecto a TF1.x (identifique si la divergencia surge en la pasada hacia delante del modelo y no en otra parte del mismo).

## Paso 1: Verifique que las variables sólo se crean una vez

Lo primero que debe comprobar es que ha construido correctamente el modelo de tal forma que reutiliza las variables en cada llamada en lugar de crear y usar nuevas variables cada vez de forma accidental. Por ejemplo, si su modelo crea una nueva capa Keras o llama a `tf.Variable` en cada llamada de pasada, lo más probable es que esté fallando en la captura de variables y creando otras nuevas cada vez.

A continuación se muestran dos ámbitos del administrador de contexto que puede usar para detectar cuándo su modelo está creando nuevas variables y depurar qué parte del modelo lo está haciendo.

In [ ]:
@contextmanager
def assert_no_variable_creations():
  """Assert no variables are created in this context manager scope."""
  def invalid_variable_creator(next_creator, **kwargs):
    raise ValueError("Attempted to create a new variable instead of reusing an existing one. Args: {}".format(kwargs))

  with tf.variable_creator_scope(invalid_variable_creator):
    yield

@contextmanager
def catch_and_raise_created_variables():
  """Raise all variables created within this context manager scope (if any)."""
  created_vars = []
  def variable_catcher(next_creator, **kwargs):
    var = next_creator(**kwargs)
    created_vars.append(var)
    return var

  with tf.variable_creator_scope(variable_catcher):
    yield
  if created_vars:
    raise ValueError("Created vars:", created_vars)

El primer ámbito (`assert_no_variable_creations()`) lanzará un error inmediatamente cuando intente crear una variable dentro del ámbito. Esto le permite inspeccionar el stacktrace (y usar depuración interactiva) para averiguar exactamente qué líneas de código crearon una variable en lugar de reutilizar una existente.

El segundo ámbito (`catch_and_raise_created_variables()`) lanzará una excepción al final del ámbito si se ha creado alguna variable. Esta excepción incluirá la lista de todas las variables creadas en el ámbito. Esto es útil para averiguar cuál es el conjunto de todas las ponderaciones que está creando su modelo en caso de que pueda detectar patrones generales. Sin embargo, es menos útil para identificar las líneas exactas de código en las que se crearon esas variables.

Use los dos ámbitos siguientes para verificar que la capa InceptionResnetV2 basada en shim no crea ninguna variable nueva después de la primera llamada (presumiblemente las reutiliza).

In [ ]:
model = InceptionResnetV2(1000)
height, width = 299, 299
num_classes = 1000

inputs = tf.ones( (1, height, width, 3))
# Create all weights on the first call
model(inputs)

# Verify that no new weights are created in followup calls
with assert_no_variable_creations():
  model(inputs)
with catch_and_raise_created_variables():
  model(inputs)

En el ejemplo siguiente, observe cómo funcionan estos decoradores en una capa que crea incorrectamente nuevas ponderaciones cada vez en lugar de reutilizar las existentes.

In [ ]:
class BrokenScalingLayer(tf.keras.layers.Layer):
  """Scaling layer that incorrectly creates new weights each time:"""

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    var = tf.Variable(initial_value=2.0)
    bias = tf.Variable(initial_value=2.0, name='bias')
    return inputs * var + bias

In [ ]:
model = BrokenScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

try:
  with assert_no_variable_creations():
    model(inputs)
except ValueError as err:
  import traceback
  traceback.print_exc()


In [ ]:
model = BrokenScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

try:
  with catch_and_raise_created_variables():
    model(inputs)
except ValueError as err:
  print(err)

Puede arreglar la capa asegurándose de que sólo crea las ponderaciones una vez y luego las reutiliza cada vez.

In [ ]:
class FixedScalingLayer(tf.keras.layers.Layer):
  """Scaling layer that incorrectly creates new weights each time:"""
  def __init__(self):
    super().__init__()
    self.var = None
    self.bias = None

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    if self.var is None:
      self.var = tf.Variable(initial_value=2.0)
      self.bias = tf.Variable(initial_value=2.0, name='bias')
    return inputs * self.var + self.bias

model = FixedScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

with assert_no_variable_creations():
  model(inputs)
with catch_and_raise_created_variables():
  model(inputs)

### Solución de problemas

Aquí tiene algunas razones comunes por las que su modelo podría estar creando accidentalmente nuevas ponderaciones en lugar de reutilizar las existentes:

1. Utiliza una llamada explícita `tf.Variable` sin reutilizar `tf.Variables` ya creadas. Solucione esto comprobando primero si no se ha creado y luego reutilizando las existentes.
2. Crea una capa o modelo Keras directamente en la pasada hacia delante cada vez (a diferencia de `tf.compat.v1.layers`). Solucione esto comprobando primero si no se ha creado y luego reutilizando las existentes.
3. Se construye sobre `tf.compat.v1.layers` pero no asigna a todas las `compat.v1.layers` un nombre explícito ni encapsula su uso de `compat.v1.layer` dentro de un `variable_scope` con nombre, provocando que los nombres de capa autogenerados se incrementen en cada llamada al modelo. Solucione esto colocando un `tf.compat.v1.variable_scope` con nombre dentro de su método decorado con shim que encapsule todo su uso de `tf.compat.v1.layers`.

## Paso 2: Verifique que el recuento, los nombres y las formas de las variables coinciden

El segundo paso consiste en asegurarse de que su capa ejecutada en TF2 crea el mismo número de ponderaciones, con las mismas formas, que lo que hace el código correspondiente en TF1.x.

Puede hacer una mezcla de revisarlos manualmente para comprobar que coinciden, y hacer las comprobaciones mediante programación en una prueba de unidad, como se muestra a continuación.

In [ ]:
# Build the forward pass inside a TF1.x graph, and 
# get the counts, shapes, and names of the variables
graph = tf.Graph()
with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
  height, width = 299, 299
  num_classes = 1000
  inputs = tf.ones( (1, height, width, 3))

  out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

  tf1_variable_names_and_shapes = {
      var.name: (var.trainable, var.shape) for var in tf.compat.v1.global_variables()}
  num_tf1_variables = len(tf.compat.v1.global_variables())

Después, haga lo mismo para la capa encapsulada con shim en TF2. Observe que también se llama al modelo varias veces antes de extraer las ponderaciones. Esto se hace para comprobar eficazmente la reutilización de variables.

In [ ]:
height, width = 299, 299
num_classes = 1000

model = InceptionResnetV2(num_classes)
# The weights will not be created until you call the model

inputs = tf.ones( (1, height, width, 3))
# Call the model multiple times before checking the weights, to verify variables
# get reused rather than accidentally creating additional variables
out, endpoints = model(inputs, training=False)
out, endpoints = model(inputs, training=False)

# Grab the name: shape mapping and the total number of variables separately,
# because in TF2 variables can be created with the same name
num_tf2_variables = len(model.variables)
tf2_variable_names_and_shapes = {
    var.name: (var.trainable, var.shape) for var in model.variables}

In [ ]:
# Verify that the variable counts, names, and shapes all match:
assert num_tf1_variables == num_tf2_variables
assert tf1_variable_names_and_shapes == tf2_variable_names_and_shapes

La capa InceptionResnetV2 basada en shim supera esta prueba. Sin embargo, en el caso de que no coincidan, puede pasarlo por un diff (de texto u otro) para ver dónde están las diferencias.

Esto puede darle una pista sobre qué parte del modelo no se está comportando como es debido. Con ejecución eager puede usar pdb, depuración interactiva y puntos de interrupción para indagar en las partes del modelo que parecen sospechosas y depurar lo que va mal con mayor profundidad.

### Solución de problemas

- Preste mucha atención a los nombres de cualquier variable creada directamente por llamadas explícitas `tf.Variable` y capas/modelos Keras, ya que su semántica de generación de nombres de variables puede diferir ligeramente entre los grafos TF1.x y la funcionalidad TF2, como ejecución eager y `tf.function`, incluso si todo lo demás funciona correctamente. Si este es su caso, ajuste su prueba para tener en cuenta cualquier semántica de nomenclatura ligeramente diferente.

- A veces puede encontrar que las `tf.Variable`s, `tf.keras.layers.Layer`s, o `tf.keras.Model`s creadas en la pasada hacia delante de su bucle de entrenamiento faltan en su lista de variables TF2 incluso si fueron capturadas por la recolección de variables en TF1.x. Solucione esto asignando las variables/capas/modelos que su pasada hacia delante crea a atributos de instancia en su modelo. Para más información, consulte [esta sección](https://www.tensorflow.org/guide/keras/custom_layers_and_models).

## Paso 3: Restablecer todas las variables, comprobar la equivalencia numérica con toda la aleatoriedad desactivada

El siguiente paso es verificar la equivalencia numérica tanto para las salidas reales como para el seguimiento de la pérdida de regularización cuando se fija el modelo de forma que no haya generación de números aleatorios (como durante la inferencia).

La forma exacta de hacerlo puede depender de su modelo específico, pero en la mayoría de los modelos (como éste), puede hacerlo de la siguiente manera:

1. Inicializar las ponderaciones al mismo valor sin aleatoriedad. Esto puede hacerse restableciéndolos a un valor fijo después de haberlos creado.
2. Ejecutar el modelo en modo de inferencia para evitar la activación de capas abandonadas que pueden ser fuentes de aleatoriedad.

El siguiente código demuestra cómo puede comparar los resultados de TF1.x y TF2 de esta forma.

In [ ]:
graph = tf.Graph()
with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
  height, width = 299, 299
  num_classes = 1000
  inputs = tf.ones( (1, height, width, 3))

  out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

  # Rather than running the global variable initializers,
  # reset all variables to a constant value
  var_reset = tf.group([var.assign(tf.ones_like(var) * 0.001) for var in tf.compat.v1.global_variables()])
  sess.run(var_reset)

  # Grab the outputs & regularization loss
  reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
  tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
  tf1_output = sess.run(out)

print("Regularization loss:", tf1_regularization_loss)
tf1_output[0][:5]

Obtenga los resultados de TF2.

In [ ]:
height, width = 299, 299
num_classes = 1000

model = InceptionResnetV2(num_classes)

inputs = tf.ones((1, height, width, 3))
# Call the model once to create the weights
out, endpoints = model(inputs, training=False)

# Reset all variables to the same fixed value as above, with no randomness
for var in model.variables:
  var.assign(tf.ones_like(var) * 0.001)
tf2_output, endpoints = model(inputs, training=False)

# Get the regularization loss
tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)
tf2_output[0][:5]

In [ ]:
# Create a dict of tolerance values
tol_dict={'rtol':1e-06, 'atol':1e-05}

In [ ]:
# Verify that the regularization loss and output both match
# when we fix the weights and avoid randomness by running inference:
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

Los números coinciden entre TF1.x y TF2 cuando se eliminan las fuentes de aleatoriedad, y la capa `InceptionResnetV2` compatible con TF2 supera la prueba.

Si observa que los resultados divergen en sus propios modelos, puede usar la impresión o pdb y la depuración interactiva para identificar dónde y por qué empiezan a divergir los resultados. La ejecución eager puede facilitarle esto considerablemente. También puede usar un enfoque de ablación para ejecutar sólo pequeñas partes del modelo en entradas intermedias fijas y aislar dónde se produce la divergencia.

Convenientemente, muchas redes delgadas (y otros modelos) también exponen puntos finales intermedios que puede sondear.

## Paso 4: Alinear la generación de números aleatorios, comprobar la equivalencia numérica en el entrenamiento y la inferencia

El paso final es verificar que el modelo TF2 coincida numéricamente con el modelo TF1.x, incluso cuando se tiene en cuenta la generación de números aleatorios en la inicialización de variables y en la propia pasada hacia adelante (tales como capas abandonadas durante la pasada hacia adelante).

Puede hacerlo usando la siguiente herramienta de pruebas para hacer coincidir la semántica de generación de números aleatorios entre grafos/sesiones TF1.x y ejecución eager.

Los grafos/sesiones heredados de TF1 y la ejecución eager de TF2 usan una semántica de generación de números aleatorios con estado diferente.

En `tf.compat.v1.Session`s, si no se especifican semillas, la generación de números aleatorios depende de cuántas operaciones hay en el grafo en el momento en que se añade la operación aleatoria y de cuántas veces se ejecuta el grafo. En ejecución eager, la generación de números aleatorios con estado depende de la semilla global, la semilla aleatoria de la operación y cuántas veces se ejecuta la operación con la semilla aleatoria dada. Consulte `tf.random.set_seed` para obtener más información.

La siguiente clase [`v1.keras.utils.DeterministicRandomTestTool`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/keras/utils/DeterministicRandomTestTool) ofrece un administrador de contexto `scope()` que puede hacer que las operaciones aleatorias con estado usen la misma semilla en ambos grafos/sesiones TF1 y ejecución eager.

La herramienta ofrece dos modos de prueba:

1. `constant` que usa la misma semilla para cada operación sin importar cuántas veces haya sido llamada y,
2. `num_random_ops` que usa el número de operaciones aleatorias con estado observadas previamente como semilla de operación.

Esto se aplica tanto a las operaciones aleatorias con estado usadas para crear e inicializar variables, como a las operaciones aleatorias con estado usadas en el cálculo (como para las capas abandonadas).

Genere tres tensores aleatorios para mostrar cómo usar esta herramienta para hacer coincidir la generación de números aleatorios con estado entre sesiones y ejecución eager.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    a = tf.random.uniform(shape=(3,1))
    a = a * 3
    b = tf.random.uniform(shape=(3,3))
    b = b * 3
    c = tf.random.uniform(shape=(3,3))
    c = c * 3
    graph_a, graph_b, graph_c = sess.run([a, b, c])

graph_a, graph_b, graph_c

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  c = tf.random.uniform(shape=(3,3))
  c = c * 3

a, b, c

In [ ]:
# Demonstrate that the generated random numbers match
np.testing.assert_allclose(graph_a, a.numpy(), **tol_dict)
np.testing.assert_allclose(graph_b, b.numpy(), **tol_dict)
np.testing.assert_allclose(graph_c, c.numpy(), **tol_dict)

Sin embargo, observe que en el modo `constant`, como `b` y `c` se generaron con la misma semilla y tienen la misma forma, tendrán exactamente los mismos valores.

In [ ]:
np.testing.assert_allclose(b.numpy(), c.numpy(), **tol_dict)

### Orden de trazado

Si le preocupa que algunos números aleatorios que coincidan en el modo `constant` reduzcan la confianza en su prueba de equivalencia numérica (por ejemplo, si varias ponderaciones adoptan las mismas inicializaciones), puede usar el modo `num_random_ops` para evitarlo. En el modo `num_random_ops`, los números aleatorios generados dependerán del orden de las ops aleatorias en el programa.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    a = tf.random.uniform(shape=(3,1))
    a = a * 3
    b = tf.random.uniform(shape=(3,3))
    b = b * 3
    c = tf.random.uniform(shape=(3,3))
    c = c * 3
    graph_a, graph_b, graph_c = sess.run([a, b, c])

graph_a, graph_b, graph_c

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  c = tf.random.uniform(shape=(3,3))
  c = c * 3

a, b, c

In [ ]:
# Demonstrate that the generated random numbers match
np.testing.assert_allclose(graph_a, a.numpy(), **tol_dict)
np.testing.assert_allclose(graph_b, b.numpy(), **tol_dict )
np.testing.assert_allclose(graph_c, c.numpy(), **tol_dict)

In [ ]:
# Demonstrate that with the 'num_random_ops' mode,
# b & c took on different values even though
# their generated shape was the same
assert not np.allclose(b.numpy(), c.numpy(), **tol_dict)

Sin embargo, tenga en cuenta que en este modo la generación aleatoria es sensible al orden del programa, por lo que los siguientes números aleatorios generados no coinciden.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3

assert not np.allclose(a.numpy(), a_prime.numpy())
assert not np.allclose(b.numpy(), b_prime.numpy())

Para permitir depurar variaciones debidas al orden de trazado, `DeterministicRandomTestTool` en modo `num_random_ops` permite ver cuántas operaciones aleatorias se han trazado con la propiedad `operation_seed`.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  print(random_tool.operation_seed)
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  print(random_tool.operation_seed)
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  print(random_tool.operation_seed)

Si necesita tener en cuenta la variación del orden de trazado en sus pruebas, puede incluso configurar la `operation_seed` de autoincremento de forma explícita. Por ejemplo, puede usarlo para que la generación de números aleatorios coincida con dos órdenes de programa diferentes.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  print(random_tool.operation_seed)
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  print(random_tool.operation_seed)
  b = tf.random.uniform(shape=(3,3))
  b = b * 3

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  random_tool.operation_seed = 1
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  random_tool.operation_seed = 0
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3

np.testing.assert_allclose(a.numpy(), a_prime.numpy(), **tol_dict)
np.testing.assert_allclose(b.numpy(), b_prime.numpy(), **tol_dict)


Sin embargo, `DeterministicRandomTestTool` no permite reutilizar semillas de operación ya en uso, así que asegúrese de que las secuencias autoincrementadas no puedan superponerse. Esto es porque ejecución eager genera números diferentes para usos sucesivos de la misma semilla de operación mientras que los grafos y sesiones TF1 no lo hacen, por lo que lanzar un error ayuda a conservar alineadas la generación de números aleatorios por estado de sesiones y eager.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  random_tool.operation_seed = 1
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  random_tool.operation_seed = 0
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3
  try:
    c = tf.random.uniform(shape=(3,1))
    raise RuntimeError("An exception should have been raised before this, " +
                     "because the auto-incremented operation seed will " +
                     "overlap an already-used value")
  except ValueError as err:
    print(err)


### Verificar la inferencia

Ahora puede usar `DeterministicRandomTestTool` para asegurarse de que el modelo `InceptionResnetV2` coincide en la inferencia, incluso cuando se usa la inicialización de ponderación aleatoria. Para una condición de prueba más fuerte debido a la coincidencia del orden del programa, use el modo `num_random_ops`.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Grab the outputs & regularization loss
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  model = InceptionResnetV2(num_classes)

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=False)

  # Grab the regularization loss as well
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool:
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

### Verificar el entrenamiento

Dado que `DeterministicRandomTestTool` funciona para *todas* las operaciones aleatorias con estado (incluyendo tanto la inicialización de las ponderaciones y la computación como las capas abandonadas), puede usarlo para verificar que los modelos coinciden también en el modo de entrenamiento. Puede volver a usar el modo `num_random_ops` porque el orden de programación de las ops aleatorias con estado coincide.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=True)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Grab the outputs & regularization loss
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  model = InceptionResnetV2(num_classes)

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=True)

  # Grab the regularization loss as well
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

Ahora ha verificado que el modelo `InceptionResnetV2` que funciona con ejecución eager y decoradores en torno a `tf.keras.layers.Layer` coincide numéricamente con la red delgada que se ejecuta en los grafos y sesiones de TF1.

Nota: Cuando utilice la `DeterministicRandomTestTool` en modo `num_random_ops`, se sugiere que use y llame directamente al decorador del método `tf.keras.layers.Layer` al comprobar la equivalencia numérica. Incorporarlo en un modelo funcional Keras o en otros modelos Keras puede producir diferencias en el orden de trazado de las operaciones aleatorias con estado que pueden ser difíciles de razonar o de hacer coincidir exactamente al comparar grafos/sesiones TF1.x y ejecución eager.

Por ejemplo, llamar directamente a la capa `InceptionResnetV2` con `training=True` intercala la inicialización de variables con el orden de abandono según el orden de creación de la red.

Por otra parte, poner primero el decorador `tf.keras.layers.Layer` en un modelo funcional Keras y sólo entonces llamar al modelo con `training=True` es equivalente a inicializar todas las variables y luego usar la capa abandonada. Esto produce un orden de trazado diferente y un conjunto diferente de números aleatorios.

Sin embargo, el `mode='constant'` por default, no es sensible a estas diferencias en el orden de trazado y pasará sin trabajo extra incluso cuando se incorpore la capa en un modelo funcional Keras.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=True)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Get the outputs & regularization losses
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  keras_input = tf.keras.Input(shape=(height, width, 3))
  layer = InceptionResnetV2(num_classes)
  model = tf.keras.Model(inputs=keras_input, outputs=layer(keras_input))

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=True)

  # Get the regularization loss
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

## Paso 3b o 4b (opcional): Pruebas con puntos de verificación preexistentes

Tras el paso 3 o el paso 4 anteriores, puede ser útil ejecutar sus pruebas de equivalencia numérica al partir de puntos de verificación preexistentes basados en nombres, si dispone de algunos. Esto puede comprobar tanto que la carga de sus puntos de verificación heredados funciona correctamente como que el propio modelo funciona correctamente. La guía [Reutilización de puntos de verificación TF1.x](./migrating_checkpoints.ipynb) cubre cómo reutilizar sus puntos de verificación TF1.x preexistentes y transferirlos a los puntos de verificación TF2.


## Pruebas adicionales y resolución de problemas

A medida que vaya añadiendo más pruebas de equivalencia numérica, también puede optar por añadir una prueba que verifique que el cálculo del gradiente (o incluso las actualizaciones del optimizador) coinciden.

La retropropagación y el cálculo de gradientes son más propensos a las inestabilidades numéricas en punto flotante que los pases hacia delante del modelo. Esto significa que a medida que sus pruebas de equivalencia cubren más partes no aisladas de su entrenamiento, puede empezar a ver diferencias numéricas no triviales entre la ejecución totalmente eager y sus grafos TF1. Esto puede deberse a las optimizaciones de grafos de TensorFlow que hacen cosas como reemplazar subexpresiones en un grafo con menos operaciones matemáticas.

Para aislar si es posible que éste sea el caso, puede comparar su código TF1 con el cómputo TF2 que ocurre dentro de una `tf.function` (que aplica pases de optimización de grafos como su grafo TF1) en lugar de con un cómputo puramente eager. Alternativamente, puede intentar usar `tf.config.optimizer.set_experimental_options` para desactivar pases de optimización como `"arithmetic_optimization"` antes de su cómputo TF1 para ver si el resultado acaba numéricamente más cerca de los resultados de su cómputo TF2. En sus ejecuciones reales de entrenamiento se recomienda usar `tf.function` con los pases de optimización habilitados por razones de rendimiento, pero puede encontrar útil deshabilitarlos en sus pruebas de unidad de equivalencia numérica.

Del mismo modo, también puede encontrar que los optimizadores `tf.compat.v1.train` y los optimizadores TF2 tienen unas propiedades numéricas en punto flotante ligeramente diferentes a las de los optimizadores TF2, aunque las fórmulas matemáticas que estén representando sean las mismas. Es menos probable que esto suponga un problema en sus ejecuciones de entrenamiento, pero puede requerir una tolerancia numérica más alta en las pruebas de unidad de equivalencia.